In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import json


kaggle_input_dir='/kaggle/input/'


# Filter columns to keep only the relevant metrics
columns_to_keep = [
    'Virtual Memory Load [%]',
    'Physical Memory Load [%]',
    # 'GPU Memory Usage [%]',
    'Total CPU Usage [%]',
    'GPU Total Usage [%]',
    'Total Activity [%]',
    # 'GPU Computing Usage [%]',
    # 'GPU Core Load [%]',
    
    'GPU Power [W]',
    # 'IGPU Power [W]',
    'CPU Package Power [W]',
    # 'PCH Power [W]',
    # 'Total DRAM Power [W]'
    
    'CPU Package Temperature [°C]',
    'GPU Temperature [°C]',

    
    # 'Potenza totale CPU [W]',
    # "CPU Package Power [W]",
    # 'Potenza Core IA [W]',
    # "IA Cores Power [W]",
    # 'VR VCC Corrente (SVID IOUT) [A]',
    # "VR VCC Current (SVID IOUT) [A]"
    # 'GPU Potenza [W]',
    # "GPU Power [W]",
    # 'IGPU Potenza [W]',
    # "IGPU Power [W]"
    # 'Potenza DRAM totale [W]',
    # 'Consumo energetico resto del chip [W]',
    # 'Rest-of-Chip Power [W]',
    
]


In [2]:
def get_csv_data(input_csv_file, show_res=False):   
    
    # Open CSV file and load data into DataFrame
    data = pd.read_csv(kaggle_input_dir+input_csv_file, encoding='latin-1', decimal='.',usecols=range(331))
    # Remove leading/trailing whitespace and special characters (like BOM) from column names
    data.columns = data.columns.str.strip().str.replace('\ufeff', '', regex=False)

    if show_res:
        # Show the first few rows of the DataFrame
        print(data)
        print('-----------------------------------------------------')
        print(f"\nColumn names: {data.columns[:5].tolist()}")  # Print first 5 column names to verify
    return data

In [3]:
def filter_data(data, show_res=False):
    # Keep only columns that exist in the DataFrame
    existing_columns = [col for col in columns_to_keep if col in data.columns]
    data_filtered = data[existing_columns]
    
    # Convert all columns to numeric values, handling errors
    for col in data_filtered.columns:
        data_filtered[col] = pd.to_numeric(data_filtered[col], errors='coerce')
    
    # Replace NaN values with 0
    data_filtered = data_filtered.fillna(0)

    if show_res:
        print(data_filtered)
        print("-------------------")
        print(data_filtered.dtypes)
    return data_filtered

In [4]:
def show_filtered_data(data_filtered):
    # Plot each metric in a separate subplot for better readability
    # Plot each metric in a separate figure
    for column in data_filtered.columns:
        plt.figure(figsize=(12, 4))
        plt.plot(data_filtered.index, data_filtered[column], label=column)
        plt.ylabel('Power (W)')
        plt.xlabel('Time (Index)')
        plt.title(column)
        plt.grid(True)
        # Show only 10 evenly spaced x-axis labels
        step = max(1, len(data_filtered) // 10)
        plt.xticks(np.arange(0, len(data_filtered), step))
        plt.tight_layout()
    
    plt.show()

In [5]:
def get_request_df_and_data_filtered(data_filtered, data, input_json_file, show_res=False):
    # Convert the first two time-related columns to datetime
    # Combine Date and Time columns to create a datetime column
    data_filtered['timestamp'] = pd.to_datetime(
        data['Date'] + ' ' + data['Time'],
        format='%d.%m.%Y %H:%M:%S.%f',
        errors='coerce'
    )
    if show_res:
        print(f"\nCSV time range (corrected): {data_filtered['timestamp'].min()} to {data_filtered['timestamp'].max()}")
    
    # Load JSON benchmark data
    json_file = kaggle_input_dir+input_json_file
    with open(json_file, 'r', encoding='utf-8') as f:
        benchmark_data = json.load(f)
    
    # Extract request data and timestamps from JSON
    request_events = []
    if 'results' in benchmark_data:
        for entry in benchmark_data['results']:
            request_events.append({
                'timestamp_send': pd.to_datetime(entry['timestamp_send']),
                'timestamp_response': pd.to_datetime(entry['timestamp_response']),
                'total_tokens': entry.get('total_tokens', 0),
                'elapsed_time': entry.get('elapsed_time_seconds', 0),
                'size_category': entry.get('size_category', 'unknown')
            })
    
    # Convert to DataFrame
    request_df = pd.DataFrame(request_events)
    
    if len(request_df) > 0:
        if show_res:
            print(f"\nJSON time range: {request_df['timestamp_send'].min()} to {request_df['timestamp_send'].max()}")
            print(f"Total requests: {len(request_df)}")
        
        # Get the first timestamp_send and last timestamp_response from JSON
        first_json_timestamp = request_df['timestamp_send'].min()
        last_json_timestamp = request_df['timestamp_response'].max()
        if show_res:
            print(f"\nFirst timestamp_send in JSON: {first_json_timestamp}")
            print(f"Last timestamp_response in JSON: {last_json_timestamp}")
        
        # Filter CSV data to show only from first request send to last response
        data_filtered = data_filtered[
            (data_filtered['timestamp'] >= first_json_timestamp) & 
            (data_filtered['timestamp'] <= last_json_timestamp)
        ].copy()
        if show_res:
            print(f"CSV data filtered to {len(data_filtered)} rows (from first request to last response)")
        
        # For each request, find the closest CSV measurement
        request_df['csv_index'] = request_df['timestamp_send'].apply(
            lambda req_time: (data_filtered['timestamp'] - req_time).abs().idxmin() 
            if pd.notna(data_filtered['timestamp']).any() else None
        )
        
        # Get the closest timestamp for each request
        request_df['closest_csv_time'] = request_df['csv_index'].apply(
            lambda idx: data_filtered.loc[idx, 'timestamp'] if idx is not None else None
        )
        
        request_df['time_diff_seconds'] = (request_df['timestamp_send'] - request_df['closest_csv_time']).dt.total_seconds()

        if show_res:
            print(f"\nTime difference statistics (seconds):")
            print(request_df['time_diff_seconds'].describe())
    return (request_df, data_filtered)

In [6]:
def show_requests(request_df,data_filtered):
    # Define colors for each category
    category_colors = {
        'short': 'green',
        'medium': 'skyblue',
        'long': 'red',
    }

    existing_columns = [col for col in columns_to_keep if col in data_filtered.columns]
    # Plot power metrics with request time bands
    for column in existing_columns:
        plt.figure(figsize=(14, 6))
        
        # Plot power data vs timestamp
        ax1 = plt.gca()
        ax1.plot(data_filtered['timestamp'], data_filtered[column], label=column, color='blue', linewidth=1.5)
        ax1.set_ylabel('Power (W)', color='blue', fontsize=12)
        ax1.set_xlabel('Time', fontsize=12)
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1.grid(True, alpha=0.3)
        
        # Add time bands for each request
        if len(request_df) > 0:
            # Track which categories have been added to legend
            added_to_legend = set()
            
            for idx, row in request_df.iterrows():
                category = row['size_category']
                color = category_colors.get(category, 'lightgray')
                label = f'{category.capitalize()} Request' if category not in added_to_legend else None
                
                # Draw vertical span from timestamp_send to timestamp_response
                ax1.axvspan(row['timestamp_send'], row['timestamp_response'], 
                           alpha=0.3, color=color, label=label)
                
                if label:
                    added_to_legend.add(category)
        
        plt.title(f'{column} with LLM Request Time Periods', fontsize=14, fontweight='bold')
        ax1.legend(loc='lower left')
        
        # Format x-axis for better readability
        plt.gcf().autofmt_xdate()
        plt.tight_layout()
    
    plt.show()

In [7]:
def get_result(request_df,data_filtered):
    data_filtered['csv_index']=data_filtered.index
    result=data_filtered.merge(request_df, on='csv_index', how='left')
    result['GPU energy [Joule]']=result['GPU Power [W]']*result['elapsed_time']
    result['CPU Package Power energy [Joule]']=result['CPU Package Power [W]']*result['elapsed_time']
    result['Total energy [Joule]']=result['GPU energy [Joule]']+result['CPU Package Power energy [Joule]']
    return result

In [8]:
def get_data(input_csv_file, input_json_files_list, show_plots=False, show_intermediate=False):
    result=pd.DataFrame()
    for input_json_file in input_json_files_list:
        data=get_csv_data(input_csv_file,show_intermediate)
        data_filtered=filter_data(data,show_intermediate)
        if show_plots:
            show_filtered_data(data_filtered)
        request_df,data_filtered=get_request_df_and_data_filtered(data_filtered,data,input_json_file,show_intermediate)
        if show_plots:
            show_requests(request_df,data_filtered)
        result = pd.concat([result,get_result(request_df,data_filtered)], ignore_index=True)
    return result

In [9]:
def get_mean_by_type(result):
    return result.groupby('size_category').mean().drop(['csv_index','timestamp',
                                             'timestamp_send','timestamp_response','closest_csv_time'],axis=1)

In [10]:
local_server_result=get_data('local-server-pc-performance-indicators/local.CSV',
        [
        'local-server-pc-performance-indicators/benchmark_liquid_lfm2.5-1.2b_20260125_123635.json',
         'local-server-pc-performance-indicators/benchmark_llama3.1-8b-chinese-chat_20260125_123635.json',
         'local-server-pc-performance-indicators/benchmark_qwen_qwen3-4b-thinking-2507_20260125_123635.json',
         'local-server-pc-performance-indicators/benchmark_openai_gpt-oss-20b_20260125_123635.json'
        ])

remote_normal_client_result=get_data('remote-normal-pc-performance-indicators/net_normal_client.CSV',
        [
            'remote-normal-pc-performance-indicators/benchmark_liquid_lfm2.5-1.2b_20260126_004413.json',
            'remote-normal-pc-performance-indicators/benchmark_llama3.1-8b-chinese-chat_20260126_004413.json',
            'remote-normal-pc-performance-indicators/benchmark_openai_gpt-oss-20b_20260126_004413.json',
            'remote-normal-pc-performance-indicators/benchmark_qwen_qwen3-4b-thinking-2507_20260126_004413.json'
        ])

remote_normal_server_result=get_data('remote-normal-pc-performance-indicators/net_server_normal.CSV',
        [
            'remote-normal-pc-performance-indicators/benchmark_liquid_lfm2.5-1.2b_20260126_004413.json',
            'remote-normal-pc-performance-indicators/benchmark_llama3.1-8b-chinese-chat_20260126_004413.json',
            'remote-normal-pc-performance-indicators/benchmark_openai_gpt-oss-20b_20260126_004413.json',
            'remote-normal-pc-performance-indicators/benchmark_qwen_qwen3-4b-thinking-2507_20260126_004413.json'
        ])

remote_medium_client_result=get_data('remote-medium-pc-performance-indicators/net_medium_client.CSV',
        [
            'remote-medium-pc-performance-indicators/benchmark_liquid_lfm2.5-1.2b_20260127_125007.json',
            'remote-medium-pc-performance-indicators/benchmark_llama3.1-8b-chinese-chat_20260127_125007.json',
            'remote-medium-pc-performance-indicators/benchmark_openai_gpt-oss-20b_20260127_125007.json',
            'remote-medium-pc-performance-indicators/benchmark_qwen_qwen3-4b-thinking-2507_20260127_125007.json'
        ])

remote_medium_server_result=get_data('remote-medium-pc-performance-indicators/net_server_medium.CSV',
        [
            'remote-medium-pc-performance-indicators/benchmark_liquid_lfm2.5-1.2b_20260127_125007.json',
            'remote-medium-pc-performance-indicators/benchmark_llama3.1-8b-chinese-chat_20260127_125007.json',
            'remote-medium-pc-performance-indicators/benchmark_openai_gpt-oss-20b_20260127_125007.json',
            'remote-medium-pc-performance-indicators/benchmark_qwen_qwen3-4b-thinking-2507_20260127_125007.json'
        ])

remote_bad_client_result=get_data('remote-bad-pc-performance-indicators/net_bad_client.CSV',
        [
            'remote-bad-pc-performance-indicators/benchmark_liquid_lfm2.5-1.2b_20260127_151031.json',
            'remote-bad-pc-performance-indicators/benchmark_llama3.1-8b-chinese-chat_20260127_151031.json',
            'remote-bad-pc-performance-indicators/benchmark_openai_gpt-oss-20b_20260127_151031.json',
            'remote-bad-pc-performance-indicators/benchmark_qwen_qwen3-4b-thinking-2507_20260127_151031.json'
        ])

remote_bad_server_result=get_data('remote-bad-pc-performance-indicators/net_server_bad.CSV',
        [
            'remote-bad-pc-performance-indicators/benchmark_liquid_lfm2.5-1.2b_20260127_151031.json',
            'remote-bad-pc-performance-indicators/benchmark_llama3.1-8b-chinese-chat_20260127_151031.json',
            'remote-bad-pc-performance-indicators/benchmark_openai_gpt-oss-20b_20260127_151031.json',
            'remote-bad-pc-performance-indicators/benchmark_qwen_qwen3-4b-thinking-2507_20260127_151031.json'
        ])



# analyse('remote-normal-pc-performance-indicators/net_normal_client.CSV',
        # 'remote-normal-pc-performance-indicators/benchmark_liquid_lfm2.5-1.2b_20260126_004413.json')

/tmp/ipykernel_55/466890706.py:4: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,281,282,283,284,285,286,287,288,289,290,294,295,296,297,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330) have mixed types. Specify dtype option on import

In [11]:
local_server_mean=get_mean_by_type(local_server_result)
remote_normal_client_mean=get_mean_by_type(remote_normal_client_result)
remote_normal_server_mean=get_mean_by_type(remote_normal_server_result)
remote_medium_client_mean=get_mean_by_type(remote_medium_client_result)
remote_medium_server_mean=get_mean_by_type(remote_medium_server_result)
remote_bad_client_mean=get_mean_by_type(remote_bad_client_result)
remote_bad_server_mean=get_mean_by_type(remote_bad_server_result)


In [12]:
local_server_mean

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long,65.929167,79.340833,18.2750,2.550833,36.615592,23.122542,67.325000,1427.550000,37.381783,-0.022174,1085.346312,1184.480005,2269.826316
medium,65.937500,79.574167,17.4275,1.830000,35.450517,22.489550,65.638333,825.841667,20.277542,0.000139,617.723611,613.563855,1231.287466
short,65.970833,79.617500,16.6025,2.964167,31.754183,20.954517,66.170833,204.983333,5.976475,-0.095428,177.052552,157.275996,334.328547


In [13]:
remote_normal_client_mean

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long,43.881667,48.085833,9.898333,0.175833,3.008275,8.318192,41.430833,1400.691667,36.474600,0.052364,114.405807,253.026724,367.432530
medium,43.857500,48.005833,9.318333,0.171667,2.901467,7.522100,41.460000,828.075000,19.502358,-0.046289,59.877956,132.306205,192.184161
short,43.859167,48.000833,9.570833,0.185000,2.896108,7.672858,41.437500,204.291667,4.145575,-0.014919,12.683392,29.240268,41.923661


In [14]:
remote_normal_server_mean

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long,58.207500,84.161667,21.042500,7.090833,34.801750,25.333558,73.135833,1400.691667,36.474600,0.072905,899.407874,1128.470770,2027.878644
medium,58.195000,84.035833,20.697500,7.367500,36.733108,24.657892,72.575000,828.075000,19.502358,-0.015414,540.561836,560.612703,1101.174539
short,58.235833,84.081667,22.495833,11.773333,30.289458,24.414517,71.677500,204.291667,4.145575,-0.105419,115.369915,118.284560,233.654475


In [15]:
remote_medium_client_mean

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long,61.012500,61.973333,12.350833,0.115833,0.0,7.298875,0.0,1392.850000,37.319642,-0.038860,0.0,230.475384,230.475384
medium,60.994167,62.048333,11.955833,0.207500,0.0,7.896708,0.0,832.408333,20.314842,-0.009033,0.0,139.475049,139.475049
short,61.001667,62.068333,11.655000,0.154167,0.0,8.188750,0.0,204.850000,4.949700,0.051268,0.0,35.252536,35.252536


In [16]:
remote_medium_server_mean

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long,59.565000,80.533333,16.290000,2.155000,19.788492,30.240233,67.841667,1392.850000,37.319642,0.016923,698.913692,1336.595898,2035.509591
medium,59.601667,80.629167,17.189167,3.603333,23.323133,28.599817,66.665833,832.408333,20.314842,0.008633,432.096889,713.879484,1145.976373
short,59.599167,80.674167,15.460000,2.909167,21.039233,25.928783,66.560000,204.850000,4.949700,-0.052190,93.341059,159.396927,252.737985


In [17]:
remote_bad_client_mean

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long,62.351667,58.891667,10.872500,0.135833,0.0,5.111633,0.0,1424.596639,44.928711,0.029263,0.0,224.205868,224.205868
medium,62.331667,58.986667,11.005833,0.190000,0.0,4.906808,0.0,830.974790,24.242254,-0.047236,0.0,116.813674,116.813674
short,62.324167,58.992500,10.844167,0.145000,0.0,4.883142,0.0,202.316667,8.346008,-0.084901,0.0,40.171368,40.171368


In [18]:
remote_bad_server_mean

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long,59.420833,68.501667,10.852500,1.084167,13.452150,16.939625,63.003333,1424.596639,44.928711,0.065863,553.049828,880.304357,1433.354186
medium,59.460833,68.593333,10.884167,1.065000,13.968258,16.168658,60.698333,830.974790,24.242254,-0.022128,311.976365,457.383871,769.360236
short,59.534167,68.773333,11.027500,1.189167,13.194617,16.124267,60.634167,202.316667,8.346008,-0.001801,110.850593,157.909495,268.760088


In [19]:
# remote_normal_client_result
# remote_medium_client_result
# remote_bad_client_result
# remote_normal_server_result
remote_medium_server_result
# remote_bad_server_result

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],timestamp,csv_index,timestamp_send,timestamp_response,total_tokens,elapsed_time,size_category,closest_csv_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
0,50.2,75.3,11.8,4.8,8.254,23.429,52.2,2026-01-27 12:50:15.436,16,2026-01-27 12:50:14.487980,2026-01-27 12:50:15.751760,165.0,1.264,short,2026-01-27 12:50:15.436,-0.948020,10.433056,29.614256,40.047312
1,50.3,75.3,13.8,1.8,9.021,10.572,53.1,2026-01-27 12:50:17.455,17,2026-01-27 12:50:16.753224,2026-01-27 12:50:18.102183,165.0,1.349,short,2026-01-27 12:50:17.455,-0.701776,12.169329,14.261628,26.430957
2,50.3,75.3,13.9,3.8,61.266,15.487,54.0,2026-01-27 12:50:19.456,18,2026-01-27 12:50:19.104270,2026-01-27 12:50:24.118232,165.0,5.014,short,2026-01-27 12:50:19.456,-0.351730,307.187724,77.651818,384.839542
3,50.3,75.3,11.5,3.6,7.773,14.607,53.4,2026-01-27 12:50:21.452,19,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,50.3,75.1,13.5,2.6,5.891,7.807,52.9,2026-01-27 12:50:23.452,20,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918,52.6,70.2,11.2,0.4,48.229,26.044,73.8,2026-01-27 13:24:38.000,1045,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3919,52.5,70.1,10.5,0.4,46.905,27.384,73.8,2026-01-27 13:24:39.992,1046,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3920,52.5,70.1,12.4,0.0,48.469,27.960,73.9,2026-01-27 13:24:42.006,1047,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3921,52.5,70.1,11.3,0.6,46.791,27.420,74.0,2026-01-27 13:24:44.024,1048,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [20]:
remote_normal_server_result['size_category']=remote_normal_server_result['size_category']+'_normal'
remote_medium_server_result['size_category']=remote_medium_server_result['size_category']+'_medium'
remote_bad_server_result['size_category']=remote_bad_server_result['size_category']+'_bad'

remote_normal_client_result['size_category']=remote_normal_client_result['size_category']+'_normal'
remote_medium_client_result['size_category']=remote_medium_client_result['size_category']+'_medium'
remote_bad_client_result['size_category']=remote_bad_client_result['size_category']+'_bad'

In [21]:
server_result=pd.DataFrame()
server_result=pd.concat([remote_normal_server_result,server_result], ignore_index=True)
server_result=pd.concat([remote_medium_server_result,server_result], ignore_index=True)
server_result=pd.concat([server_result,remote_bad_server_result], ignore_index=True)

client_result=pd.DataFrame()
client_result=pd.concat([remote_normal_client_result,client_result], ignore_index=True)
client_result=pd.concat([remote_medium_client_result,client_result], ignore_index=True)
client_result=pd.concat([client_result,remote_bad_client_result], ignore_index=True)

In [22]:
get_mean_by_type(server_result)

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long_bad,59.420833,68.501667,10.852500,1.084167,13.452150,16.939625,63.003333,1424.596639,44.928711,0.065863,553.049828,880.304357,1433.354186
long_medium,59.565000,80.533333,16.290000,2.155000,19.788492,30.240233,67.841667,1392.850000,37.319642,0.016923,698.913692,1336.595898,2035.509591
long_normal,58.207500,84.161667,21.042500,7.090833,34.801750,25.333558,73.135833,1400.691667,36.474600,0.072905,899.407874,1128.470770,2027.878644
medium_bad,59.460833,68.593333,10.884167,1.065000,13.968258,16.168658,60.698333,830.974790,24.242254,-0.022128,311.976365,457.383871,769.360236
medium_medium,59.601667,80.629167,17.189167,3.603333,23.323133,28.599817,66.665833,832.408333,20.314842,0.008633,432.096889,713.879484,1145.976373
medium_normal,58.195000,84.035833,20.697500,7.367500,36.733108,24.657892,72.575000,828.075000,19.502358,-0.015414,540.561836,560.612703,1101.174539
short_bad,59.534167,68.773333,11.027500,1.189167,13.194617,16.124267,60.634167,202.316667,8.346008,-0.001801,110.850593,157.909495,268.760088
short_medium,59.599167,80.674167,15.460000,2.909167,21.039233,25.928783,66.560000,204.850000,4.949700,-0.052190,93.341059,159.396927,252.737985
short_normal,58.235833,84.081667,22.495833,11.773333,30.289458,24.414517,71.677500,204.291667,4.145575,-0.105419,115.369915,118.284560,233.654475


In [23]:
get_mean_by_type(client_result)

,Virtual Memory Load [%],Physical Memory Load [%],Total CPU Usage [%],Total Activity [%],GPU Power [W],CPU Package Power [W],GPU Temperature [°C],total_tokens,elapsed_time,time_diff_seconds,GPU energy [Joule],CPU Package Power energy [Joule],Total energy [Joule]
size_category,,,,,,,,,,,,,
long_bad,62.351667,58.891667,10.872500,0.135833,0.000000,5.111633,0.000000,1424.596639,44.928711,0.029263,0.000000,224.205868,224.205868
long_medium,61.012500,61.973333,12.350833,0.115833,0.000000,7.298875,0.000000,1392.850000,37.319642,-0.038860,0.000000,230.475384,230.475384
long_normal,43.881667,48.085833,9.898333,0.175833,3.008275,8.318192,41.430833,1400.691667,36.474600,0.052364,114.405807,253.026724,367.432530
medium_bad,62.331667,58.986667,11.005833,0.190000,0.000000,4.906808,0.000000,830.974790,24.242254,-0.047236,0.000000,116.813674,116.813674
medium_medium,60.994167,62.048333,11.955833,0.207500,0.000000,7.896708,0.000000,832.408333,20.314842,-0.009033,0.000000,139.475049,139.475049
medium_normal,43.857500,48.005833,9.318333,0.171667,2.901467,7.522100,41.460000,828.075000,19.502358,-0.046289,59.877956,132.306205,192.184161
short_bad,62.324167,58.992500,10.844167,0.145000,0.000000,4.883142,0.000000,202.316667,8.346008,-0.084901,0.000000,40.171368,40.171368
short_medium,61.001667,62.068333,11.655000,0.154167,0.000000,8.188750,0.000000,204.850000,4.949700,0.051268,0.000000,35.252536,35.252536
short_normal,43.859167,48.000833,9.570833,0.185000,2.896108,7.672858,41.437500,204.291667,4.145575,-0.014919,12.683392,29.240268,41.923661
